# Análise de Candidatos ao Vestibular da UEA

-  Análise exploratória
    - [X] Verificar existência de dados nulos
    - [X] Formatar ``datanasc`` de str para datetime
    - [X] Formatar ``classificacao`` de float para int
    - [X] Criar atributo ``idade``
    - [X] Formatar atributo ``id_referencia_grupo``
    - [ ] Ordenar (na ordem que aparece na leitura do PDF) dataframe considerando: 
        1. Por ``ano``
        2. Por ``curso_nome_referencia``
        3. Por ``id_curso_referencia``
        4. Por ``classificacao`` 


## Pré-processamento dos dados

In [85]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

from datetime import datetime, date

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### Carregando o dataset

In [86]:
data = pd.read_csv('../data/csv/candidates_info.csv', sep=',', encoding='utf-8')

In [87]:
data.head(5)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
0,6354130-0,EDIJANE PINHEIRO MARTINS,04/03/92,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019
1,6318761-2,JOEL DOS SANTOS DE SOUZA,09/03/97,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
2,6379123-4,CARINA DAS CHAGAS SEIXAS,23/10/96,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
3,6549853-4,IRLA DA SILVA PEREIRA,07/02/01,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019
4,6393350-0,IVANIR LIMA PEREIRA,11/03/91,317.0,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019


## Análise exploratória dos dados

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184389 entries, 0 to 184388
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   numinscricao           184389 non-null  object 
 1   nome                   184389 non-null  object 
 2   datanasc               184389 non-null  object 
 3   classificacao          184389 non-null  float64
 4   opcao_curso_2          184389 non-null  object 
 5   opcao_3_curso          184389 non-null  object 
 6   habilidades            184389 non-null  object 
 7   nota_final             184389 non-null  object 
 8   nota_cg                184389 non-null  object 
 9   nota_etapa_1           184389 non-null  object 
 10  disc1                  184389 non-null  int64  
 11  disc2                  184389 non-null  int64  
 12  disc3                  184389 non-null  int64  
 13  nota_ce                184389 non-null  object 
 14  nota_redacao           184389 non-nu

#### Verificando a existência de colunas com dados nulos

In [89]:
data.isnull().sum()

numinscricao             0
nome                     0
datanasc                 0
classificacao            0
opcao_curso_2            0
opcao_3_curso            0
habilidades              0
nota_final               0
nota_cg                  0
nota_etapa_1             0
disc1                    0
disc2                    0
disc3                    0
nota_ce                  0
nota_redacao             0
nota_etapa_2             0
opcao_curso_1            0
id_referencia_curso      0
nome_referencia_curso    0
ano                      0
dtype: int64

#### Formatando coluna 'datanasc' para o formato date

In [90]:
data['datanasc'] = pd.to_datetime(data['datanasc'], format='%d/%m/%y')
data.head(4)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
0,6354130-0,EDIJANE PINHEIRO MARTINS,1992-03-04,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019
1,6318761-2,JOEL DOS SANTOS DE SOUZA,1997-03-09,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
2,6379123-4,CARINA DAS CHAGAS SEIXAS,1996-10-23,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
3,6549853-4,IRLA DA SILVA PEREIRA,2001-02-07,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019


In [116]:
data['classificacao'] = data['classificacao'].astype(int)

In [118]:
data.sample(2)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
147456,2610498-9,WENDERSON DA SILVA PEIXOTO,203,2002,3002,No,"35,873","27,381","27,381",3,5,4,"24,00","20,36","44,364",1002,2,Administração - Vespertino - Manaus,2022,23
28843,6362974-7,VICTORIA CORREA LIMA,50,2015,3015,No,"71,438","64,286","64,286",10,8,9,"54,00","24,59","78,589",1015,3,Direito - Vespertino - Manaus,2019,17


## Criando novos atributos

In [91]:
def calculate_age(row) -> int:
    born = row['datanasc']
    year = row['ano']
    age = year - born.year
    return age

In [92]:
data['idade'] = data.apply(calculate_age, axis=1)
data.drop(columns=['datanasc'], axis=1, inplace=True)
data.head(5)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
0,6354130-0,EDIJANE PINHEIRO MARTINS,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019,27
1,6318761-2,JOEL DOS SANTOS DE SOUZA,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,22
2,6379123-4,CARINA DAS CHAGAS SEIXAS,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,23
3,6549853-4,IRLA DA SILVA PEREIRA,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,18
4,6393350-0,IVANIR LIMA PEREIRA,317.0,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,28


In [93]:
data.sample(8)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
129614,1351521-7,ANA LUIZA JAFRA CORDEIRO,7.0,06016,07016,No,"65,366","63,095","63,095",8,7,8,"46,00","21,64","67,636",5016,6016,Enfermagem - Integral - Manaus,2021,19
100850,1280554-8,EDUARDA MARFIZA GUIMARAES LOBATO,195.0,02015,03015,No,"44,145","27,381","27,381",7,7,5,"38,00","22,91","60,909",1015,1015,Direito - Vespertino - Manaus,2021,17
26427,6325165-5,ALBERTA KELLEN TEIXEIRA ABITBOL,44.0,02010,03010,No,"41,916","39,286","39,286",1,5,8,"28,00","16,55","44,545",1010,3010,Ciências Contábeis - Matutino - Manaus,2019,30
139515,2411812-5,EMANUEL SOARES AMBROSIO,128.0,02010,03010,No,"28,433","19,048","19,048",3,1,6,"20,00","17,82","37,818",1010,1010,Ciências Contábeis - Matutino - Manaus,2022,17
101269,1180547-1,LAISSA DA SILVA MENDONCA,115.0,02017,03017,No,"27,492","22,619","22,619",1,1,4,"12,00","20,36","32,364",1017,1017,Engenharia Civil - Integral - Manaus,2021,18
93910,8331790-2,LORRANY CAROLINE ARAUJO DA SILVA,416.0,06065,07065,No,"33,468","28,571","28,571",2,2,5,"18,00","20,36","38,364",5065,6065,Odontologia - Integral - Manaus,2020,17
75654,8437853-0,ANNA RAFAELLA BASTOS DE OLIVEIRA,158.0,02018,03018,No,"45,271","36,905","36,905",3,2,11,"32,00","21,64","53,636",1018,2018,Engenharia da Computação - Integral - Manaus,2020,18
13266,6332035-5,MARCELLA NAYARA LEMOS SOUZA,554.0,06037,07037,No,"20,169","21,429","21,429",3,1,1,"10,00","8,91","18,909",5037,7037,Licenciatura em Educação Física - Vespertino -...,2019,20


#### Padronizando as colunas referentes a opção de curso

In [94]:
data['opcao_curso_1'] = data['opcao_curso_1'].astype(str)
data['id_referencia_curso'] = data['id_referencia_curso'].astype(str)

In [95]:
data['opcao_3_curso'] = data['opcao_3_curso'].apply(lambda x: x[1:] if str(x).startswith('0') else x)
data['opcao_curso_2'] = data['opcao_curso_2'].apply(lambda x: x[1:] if str(x).startswith('0') else x)

In [96]:
data.head(8)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
0,6354130-0,EDIJANE PINHEIRO MARTINS,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019,27
1,6318761-2,JOEL DOS SANTOS DE SOUZA,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,22
2,6379123-4,CARINA DAS CHAGAS SEIXAS,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,23
3,6549853-4,IRLA DA SILVA PEREIRA,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,18
4,6393350-0,IVANIR LIMA PEREIRA,317.0,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,28
5,6612916-8,LAISA DOS SANTOS GALVAO,318.0,No,No,No,"31,046","25,000","25,000",4,2,3,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,16
6,6424859-3,ELCIA NEVES DOS SANTOS,319.0,No,No,No,"31,046","25,000","25,000",5,3,1,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,18
7,6386972-1,LAIZA DE FARIAS MAKLOUF,320.0,No,No,No,"31,004","26,190","26,190",4,3,2,"18,00","17,82","35,818",4069,4069,Odontologia - Integral - Manaus,2019,17


In [97]:
data.dtypes

numinscricao              object
nome                      object
classificacao            float64
opcao_curso_2             object
opcao_3_curso             object
habilidades               object
nota_final                object
nota_cg                   object
nota_etapa_1              object
disc1                      int64
disc2                      int64
disc3                      int64
nota_ce                   object
nota_redacao              object
nota_etapa_2              object
opcao_curso_1             object
id_referencia_curso       object
nome_referencia_curso     object
ano                        int64
idade                      int64
dtype: object

## Relacionado as informações sobre o curso com cada candidato

Seguindo o padrão observado na extração de dados, id_referencia curso é composto por **XXYYY**. Onde:
- XX: grupo relacionado ao curso
- YY: código do curso

Para conseguir comparar o dataset de candidatos com o de vagas, iremos utilizar somente as informações do grupo (XX) na coluna: 

In [101]:
data['id_referencia_curso'] = data['id_referencia_curso'].apply(lambda x: x[:1])

In [119]:
data.sample(4)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
160016,2358393-2,ERIKA ROCHA DA SILVA COSTA,601,2015,3015,No,"37,699","29,762","29,762",2,5,5,"24,00","21,64","45,636",1015,3,Direito - Vespertino - Manaus,2022,17
137854,1148706-2,GEOVANNA DUARTE SOARES,1,No,No,No,"53,437","45,238","45,238",3,6,11,"40,00","21,64","61,636",8037,8,Licenciatura em Educação Física - Vespertino -...,2021,18
116224,1281975-1,NATHALIE KELLY GOMES ROSA,27,2010,3010,No,"42,611","42,857","42,857",1,4,6,"22,00","20,36","42,364",1010,3,Ciências Contábeis - Matutino - Manaus,2021,17
125661,1180267-7,MARCELA ALVES DE LIMA,119,No,No,No,"35,022","30,952","30,952",3,3,4,"20,00","19,09","39,091",4016,4,Enfermagem - Integral - Manaus,2021,18


Agora, vamos ordenar o dataset por ano, id_referencia_curso e classificação